In [99]:
### Random Forest Forecasting Model ###

In [100]:
## Import Relevant Libraries
import pandas as pd
import numpy as np
from Forecasters import TSDatasetGenerator, RFForecaster
from Forecasters.XGBForecaster import XGBForecaster
from sklearn.metrics import confusion_matrix, plot_confusion_matrix, classification_report

In [101]:
## Change director to import custom dependencies and libraries
os.chdir('/Users/elisha/Desktop/EC4308 Week 13/EC4308 Project Code')

In [102]:
from Forecasters import TSDatasetGenerator
from Forecasters.RFForecaster import RFForecaster

In [103]:
## Load datasets
# Without DATE col
train_data = pd.read_csv('rec_train.csv').drop(columns = ['Unnamed: 0', 'DATE'])
train_data.head()

,Is_Recession,PAYEMS,FEDFUNDS,3MTB_SECONDARYMKT,GS1,GS5,GS10,CPI,DEBT_PUB,SP500,INDPRO,UNRATE,10Y3MTH_SPREAD,FEDFUNDS_ROLMEAN3,INDPRO_ROLMEAN3,UNRATE_ROLMEAN3,PAYEMS_3MTHCHANGE,10Y3MTH_SPREAD_3MTHCHANGE,INDPRO_3MTHCHANGE
0,0,275.0,0.23,0.02,0.03,0.06,0.03,-0.623053,0.00000,-4.93,0.3323,0.1,0.01,0.100000,0.286133,0.033333,0.000066,-5.722222,0.003078
1,0,399.0,0.27,-0.14,0.04,0.08,0.03,0.309598,0.00000,-1.39,0.1662,-0.1,0.17,0.170000,0.184633,0.000000,-0.000044,0.329365,-0.009071
2,0,190.0,0.13,0.30,0.20,0.20,0.21,-0.000956,0.35328,-1.14,0.1938,0.0,-0.09,0.210000,0.230767,0.000000,-0.000923,0.585165,0.003938
3,0,206.0,0.23,0.16,0.37,0.33,0.20,0.306743,0.00000,-6.50,0.0277,0.0,0.04,0.210000,0.129233,-0.033333,-0.001142,0.164835,-0.008827
4,0,136.0,-0.13,0.41,0.28,0.00,-0.04,-0.615385,0.00000,-0.54,0.3323,-0.1,-0.45,0.076667,0.184600,-0.033333,-0.004180,-2.470046,0.004638


In [104]:
test_data = pd.read_csv('rec_train.csv').drop(columns = ['Unnamed: 0', 'DATE'])
test_data.head()

,Is_Recession,PAYEMS,FEDFUNDS,3MTB_SECONDARYMKT,GS1,GS5,GS10,CPI,DEBT_PUB,SP500,INDPRO,UNRATE,10Y3MTH_SPREAD,FEDFUNDS_ROLMEAN3,INDPRO_ROLMEAN3,UNRATE_ROLMEAN3,PAYEMS_3MTHCHANGE,10Y3MTH_SPREAD_3MTHCHANGE,INDPRO_3MTHCHANGE
0,0,275.0,0.23,0.02,0.03,0.06,0.03,-0.623053,0.00000,-4.93,0.3323,0.1,0.01,0.100000,0.286133,0.033333,0.000066,-5.722222,0.003078
1,0,399.0,0.27,-0.14,0.04,0.08,0.03,0.309598,0.00000,-1.39,0.1662,-0.1,0.17,0.170000,0.184633,0.000000,-0.000044,0.329365,-0.009071
2,0,190.0,0.13,0.30,0.20,0.20,0.21,-0.000956,0.35328,-1.14,0.1938,0.0,-0.09,0.210000,0.230767,0.000000,-0.000923,0.585165,0.003938
3,0,206.0,0.23,0.16,0.37,0.33,0.20,0.306743,0.00000,-6.50,0.0277,0.0,0.04,0.210000,0.129233,-0.033333,-0.001142,0.164835,-0.008827
4,0,136.0,-0.13,0.41,0.28,0.00,-0.04,-0.615385,0.00000,-0.54,0.3323,-0.1,-0.45,0.076667,0.184600,-0.033333,-0.004180,-2.470046,0.004638


In [105]:
## Create_datasets
dataset_generator = TSDatasetGenerator()

In [106]:
## Auxiliary function to generate Classification Report
def genClassificationRep(y_true, y_pred):
    return pd.DataFrame(classification_report(y_true, y_pred, output_dict=True))

In [119]:
## Grid Search Candidate Parameters
grid_param = {
    'n_estimators': [100, 300],
    'criterion': ['gini'],
    'bootstrap': [True],
    'max_depth': [2, 4],
    'class_weight': ['balanced_subsample', 'balanced'],
    'random_state': [42]
}

In [114]:
## Auxiliary function to train models and log results for various combinations of h, k and l
def runBehemoth(grid_params, data, target_feature, n_splits, model='RF', h=1, k_range=[], l_range=[]):
    results = []
    label_index = []

    # Instantiate dataset generator
    dataset_generator = TSDatasetGenerator()

    # Loop over all combinations of k and l
    for k in k_range:
        for l in l_range:
            # Instantiate model
            if model == 'XGB':
                selected_model = XGBForecaster()
            elif model == 'RF':
                selected_model = RFForecaster()
            else:
                print('Please select a valid model: RF or XGB')
                return None

            # Label for output (h, k, l)
            res_label = "(h=%d, k=%d, l=%d)" % (h, k, l)
            print("Running grid search for the dataset with params: " + res_label)
            # Create datasets
            train = dataset_generator.fit_transform(data, target_feature, h, k, l)
            train_X = train.drop(columns=['Target Feature'])
            train_y = train['Target Feature']

            # Fit model
            fitted_model = selected_model.fit(train_X, train_y)
            fitted_model.grid_search_CV(grid_params, n_splits=n_splits)

            # Log results
            result = {}
            result['Best score (log-loss)'] = fitted_model.getBestScore()
            result['Best params'] = fitted_model.getBestParams()
            result['Best model'] = fitted_model.getBestModel()
            result['Feature Importance'] = fitted_model.getFeatureImportance()
            results.append(result)
            label_index.append(res_label)

    labels_index = pd.DataFrame(label_index, columns=['Dataset'])
    results_final = pd.DataFrame(results)
    collected_results = pd.concat([labels_index, results_final], axis=1)
    return collected_results        

In [120]:
## 1 Step Ahead Forecast
one_step_results = runBehemoth(grid_param,
                               train_data,
                               'Is_Recession',
                               300,
                               k_range=[1, 2, 3],
                               l_range=[1, 2, 3])

  0%|          | 1/300 [00:00<00:48,  6.14it/s]

Running grid search for the dataset with params: (h=1, k=1, l=1)
Dropping any existing invalid observations
Recursive cross val data generated, number of split: 300


  0%|          | 0/600 [00:00<?, ?it/s]

Recursive cross val data generated, number of split: 300


  0%|          | 1/900 [00:00<02:07,  7.04it/s]

...

  0%|          | 1/2100 [00:00<04:06,  8.51it/s]

Recursive cross val data generated, number of split: 300


  0%|          | 0/2400 [00:00<?, ?it/s]

Recursive cross val data generated, number of split: 300


100%|██████████| 2400/2400 [16:59<00:00,  2.35it/s]


In [121]:
## Write out results
one_step_results.to_csv('one_step_rf_results.csv', index=False)

In [ ]:
## 3 Step Ahead Forecast
three_step_results = runBehemoth(grid_param,
                                 train_data,
                                 'Is_Recession',
                                 300,
                                 k_range=[1, 2, 3, 4, 5, 6],
                                 l_range=[1, 2, 3, 4, 5, 6])

In [116]:
## Write out results
three_step_results.to_csv('three_step_rf_results.csv', index=False)

AttributeError: 'NoneType' object has no attribute 'to_csv'

In [ ]:
## 6 Step Ahead Forecast
six_step_results = runBehemoth(grid_param,
                               train_data,
                               'Is_Recession',
                               300,
                               k_range=[1, 2, 3, 4, 5, 6],
                               l_range=[1, 2, 3, 4, 5, 6])

In [ ]:
## Write out results
six_step_results.to_csv('six_step_rf_results.csv', index=False)

In [ ]:
## 12 Step Ahead Forecast
twelve_step_results = runBehemoth(grid_param,
                                  train_data,
                                  'Is_Recession',
                                  300,
                                  k_range=[1, 2, 3, 4, 5, 6],
                                  l_range=[1, 2, 3, 4, 5, 6])

In [ ]:
## Write out results
twelve_step_results.to_csv('twelve_step_rf_results.csv', index=False)